In [1]:
%cd ..
from resnet_quant import *

/Users/jesusnavarro/Desktop/cs282_proj


In [5]:
# Importing model
qparams = [4, 4, 'dorefa'] # [abit, qbit, quant_method] --- only dorefa has been implemented

# resnet models are 8, 14, 16, 24

# activation and weight quantization are computed slightly different.
quant_resnet = get_quant_model('resnet8', qparams)


 PreAct_ResNet_Cifar_Q(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): PreActBasicBlock_convQ(
      (act_qfn): activation_quantize_fn()
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d_Q(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (qfn): weight_quantize_fn()
      )
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d_Q(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (qfn): weight_quantize_fn()
      )
      (relu): ReLU(inplace=True)
    )
  )
  (layer2): Sequential(
    (0): PreActBasicBlock_convQ(
      (act_qfn): activation_quantize_fn()
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d_Q(
        16, 32, kernel_size=(3, 3), stride=(2, 2)

In [94]:
# Note: first and last layer are not quantized 
# looking at layer ouptuts using hooks
outputs=[]
quant_resnet = get_quant_model('resnet8', qparams)


def hook(module, input, output):
    outputs.append(output)

# Registering quantized act and 
for n, m in quant_resnet.named_modules():
    if n == 'conv1': # first layer output
        m.register_forward_hook(hook)
    elif 'qfn' in n:
        m.register_forward_hook(hook)
    elif 'fc' in n:
        m.register_forward_hook(hook)
import torch
# reset model or it will continue appending outputs 
image = torch.unsqueeze(torch.rand(size=(3, 32, 32)), dim=0)
out = quant_resnet(image)


In [95]:
# look at some values at intermediate layer outputs
# weights and acitvations are quantized but operations are still in full precision
# with the final layer also full precision

for output in outputs:
    output = output.view(1, -1)
    print(np.squeeze(output.data.numpy()[0:5]))

[-0.08691858 -0.13056552 -0.07414071 ... -0.32856515 -0.5100109
 -0.31090957]
[0.         0.         0.         ... 0.6        0.         0.73333335]
[-0.0940421  -0.03134737  0.03134739 ... -0.03134737 -0.03134737
  0.03134739]
[0.6        0.         0.4        ... 0.46666667 0.         0.2       ]
[-0.03089878  0.03089881 -0.15449388 ...  0.03089881 -0.09269633
 -0.09269633]
[0.         0.         0.         ... 1.         0.53333336 1.        ]
[-0.01912601 -0.13388206 -0.01912601 ... -0.01912601  0.01912603
 -0.01912601]
[0.         0.         0.46666667 ... 0.93333334 0.         0.        ]
[ 0.02225179  0.02225179  0.11125886 ... -0.06675531  0.02225179
  0.1557624 ]
[0. 1. 0. ... 0. 0. 0.]
[ 0.04640684  0.1082826   0.04640684 ...  0.07734472  0.04640684
 -0.01546894]
[0.         0.         0.26666668 ... 0.         0.06666667 0.8666667 ]
[ 0.01505579  0.04516736  0.07527892 ... -0.0752789  -0.04516734
 -0.01505578]
[-0.08671793  0.18297285 -0.19683298  0.16536272 -0.30727726 -0.

In [64]:
# print outputs
# simple loss function to look at gradients
loss = torch.sum(out)
print(out.grad)

None
